# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
#     print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkify
        WITH replication = {'class':'SimpleStrategy', 'replication_factor':1}
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. <br> Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data
1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

## 1st question

### Primary Key
Number of row in total dataframe is equal to the number of row when we drop the duplicate row with session_id and item_in_session. <br>
It's safe to use only `session_id`, and `item_in_session` as a primary key

In [8]:
# Here is how to derive the primary key for `session_events` table
df = pd.read_csv('event_datafile_new.csv')
print(df.shape[0] == df.drop_duplicates(subset=['sessionId','itemInSession']).shape[0])
display(df[(df['sessionId'] == 338) & (df['itemInSession'] == 4)])

True


,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
444,Faithless,Ava,F,4,Robinson,495.3073,free,"New Haven-Milford, CT",338,Music Matters (Mark Knight Dub),50


### Process

In [9]:
try:
    session.execute("DROP TABLE IF EXISTS session_events")
except Exception as e:
    print(e)
    
    
try:
    session.execute("""
    CREATE TABLE IF NOT EXISTS session_events (
    session_id int,
    item_in_session int,
    artist text,
    song text,
    length decimal,
    PRIMARY KEY (session_id, item_in_session)
    )
    """)
except Exception as e:
    print(e)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        
        query = "INSERT INTO session_events (session_id, item_in_session, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
def display_cql_df(query):
    '''
    to display the CQL result in the pandas dataframe format
    @args
    query : provide CQL query string to display
    '''
    result = pd.DataFrame([*session.execute(query)])
    display(result)

In [11]:
query = 'SELECT * FROM session_events LIMIT 5'
display_cql_df(query)

,session_id,item_in_session,artist,length,song
0,23,0,Regina Spektor,191.08526,The Calculation (Album Version)
1,23,1,Octopus Project,250.95791,All Of The Champs That Ever Lived
2,23,2,Tegan And Sara,180.06159,So Jealous
3,23,3,Dragonette,153.39057,Okay Dolores
4,23,4,Lil Wayne / Eminem,229.58975,Drop The World


In [12]:
# All rows has been ingested except header
query = 'SELECT COUNT(*) FROM session_events'
display_cql_df(query)

,count
0,6820


### Result

In [13]:
query = "SELECT artist, song, length FROM session_events WHERE session_id = 338 AND item_in_session = 4"
display_cql_df(query)

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.3073


## 2nd question

### Primary Key
Here is how to derive the primary key for `user_events` table <br>
As you can see in the below figure, the uniquenes are in the level of `user_id` and `session_id`

In [14]:
display(df[(df['userId'] == 10) & (df['sessionId'] == 182)])

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
4704,Down To The Bone,Sylvie,F,0,Cruz,333.76608,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Keep On Keepin' On,10
4705,Three Drives,Sylvie,F,1,Cruz,411.63710,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Greece 2000,10
4706,Sebastien Tellier,Sylvie,F,2,Cruz,377.73016,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Kilometer,10
4707,Lonnie Gordon,Sylvie,F,3,Cruz,181.21098,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Catch You Baby (Steve Pitron & Max Sanna Radio...,10


### Process

In [15]:
try:
    session.execute("DROP TABLE IF EXISTS user_events")
except Exception as e:
    print(e)

try:
    session.execute("""
    CREATE TABLE IF NOT EXISTS user_events (
    user_id int,
    session_id int,
    item_in_session int,
    artist text,
    song text,
    first_name text,
    last_name text,
    PRIMARY KEY ((user_id, session_id), item_in_session)
    )
    """)
except Exception as e:
    print(e)

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        
        query = "INSERT INTO user_events (user_id, session_id, item_in_session, artist, song, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

### Result

In [16]:
query = "SELECT artist, song, item_in_session, first_name, last_name FROM user_events WHERE user_id = 10 AND session_id = 182"
display_cql_df(query)

,artist,song,item_in_session,first_name,last_name
0,Down To The Bone,Keep On Keepin' On,0,Sylvie,Cruz
1,Three Drives,Greece 2000,1,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,2,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,3,Sylvie,Cruz


## 3rd Question

### Primary Key
Here is how to derive the primary key for `name_events` table <br>
As you can see in the below dataframe, the uniqueness of each row is in the level of `song`, `first_name`, `last_name`

In [17]:
display(df[df['song'] == 'All Hands Against His Own'])

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
2792,The Black Keys,Tegan,F,25,Levine,196.91057,paid,"Portland-South Portland, ME",611,All Hands Against His Own,80
5135,The Black Keys,Sara,F,31,Johnson,196.91057,paid,"Winston-Salem, NC",152,All Hands Against His Own,95
6298,The Black Keys,Jacqueline,F,50,Lynch,196.91057,paid,"Atlanta-Sandy Springs-Roswell, GA",559,All Hands Against His Own,29


### Process

In [18]:
try:
    session.execute("DROP TABLE IF EXISTS name_events")
except Exception as e:
    print(e)
    
try:
    session.execute("""
    CREATE TABLE IF NOT EXISTS name_events (
    song text,
    first_name text,
    last_name text,
    PRIMARY KEY (song, first_name, last_name)
    )
    """)
except Exception as e:
    print(e)

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
    
        query = "INSERT INTO name_events (song, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s)"
        
        session.execute(query, (line[9], line[1], line[4]))

### Result

In [19]:
query = "SELECT song, first_name, last_name FROM name_events WHERE song='All Hands Against His Own'"
display_cql_df(query)

,song,first_name,last_name
0,All Hands Against His Own,Jacqueline,Lynch
1,All Hands Against His Own,Sara,Johnson
2,All Hands Against His Own,Tegan,Levine


### Drop the tables before closing out the sessions

In [20]:
for tbl in ['session_events', 'user_events', 'name_events']:
    try:
        session.execute(f"DROP TABLE IF EXISTS {tbl}")
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [21]:
session.shutdown()
cluster.shutdown()